In [1]:
import pandas as pd

########################################## LENDO A BASE OPERACIONAL PRONTA PARA TRABALHAR
caminho = r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\BASES SEMANA PADRAO"
base_operacional_hugo = pd.read_csv(caminho+"/BASE_OPERACIONAL_TELECOB_HUGO.txt", sep=';', dtype=str,decimal=',')

base_operacional_hugo['ATRASO'] = base_operacional_hugo['ATRASO'].astype('int16')
base_operacional_hugo['SALDO_ABERTO'] = base_operacional_hugo['SALDO_ABERTO'].str.replace('.', '').str.replace(',', '.').astype(float)
base_operacional_hugo['BASE'] = base_operacional_hugo['BASE'].apply(lambda x: x.replace('Claro MÃ³vel','Claro Móvel'))
base_operacional_hugo = base_operacional_hugo.assign(FILA="")



###################### CRIANDO OS LAYOUTS PARA OS MAILINGS
preditivo_higienizado_claro = ["CPF","NOME_x","DEVEDOR","BASE","FILA","ATRASO","SALDO_ABERTO","TELEFONE_HIGIENIZADO"]
preditivo_validados_claro = ["CPF","NOME_x","DEVEDOR","BASE","FILA","ATRASO","SALDO_ABERTO",'TELEFONE',"TELEFONE_HIGIENIZADO"]
preditivo_telefones_intersic = ["CPF","NOME_x","DEVEDOR","BASE","FILA","ATRASO","SALDO_ABERTO",'TELEFONE']

base_operacional_hugo

C:\Users\marco.junior\AppData\Local\Temp\ipykernel_7920\1324496333.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  base_operacional_hugo['SALDO_ABERTO'] = base_operacional_hugo['SALDO_ABERTO'].str.replace('.', '').str.replace(',', '.').astype(float)


,CPF,DEVEDOR,NOME_x,TELEFONE,BASE_CONTRATANTE_VALIDADOS,DATA_VALIDACAO_CONTRATANTE,EMAIL,ATRASO,SALDO_ABERTO,BASE,CONTRATO,TELEFONE_HIGIENIZADO,BASE_HIGI_VALIDADOS,DATA_VALIDACAO_HIGI,EMAIL_HIGIENIZADO,FILA
0,36706306860,37085860,ALINE CAVALCANTE DE ANDRADE,11982649989,Nao_Validado,NaN,NaN,3,56.03,Claro Móvel,165188966 ...,NaN,Nao_Validado,NaN,NaN,
1,26612315873,27299068,DENILZE DORIZETI DINIZ ESCUDERO,11980175524,Nao_Validado,NaN,NaN,3,147.98,Claro Móvel,02100087501563 ...,NaN,Nao_Validado,NaN,NaN,
2,42370504838,26468203,JOSE ANTONIO DE MELO BARBOSA,11976866454,Nao_Validado,NaN,NaN,3,87.25,Claro Móvel,134393812 ...,NaN,Nao_Validado,NaN,NaN,
3,34387688874,32123055,MARCELO DA SILVA DE LIMA,18991561780,Nao_Validado,NaN,NaN,3,30.86,Claro Móvel,241681854 ...,18996165960,Sim,10/08/2023,autoe.lima@gmail.com,
4,83390910425,37141013,MARIA JOSE DO REGO OLIVEIRA,81985575771,Nao_Validado,NaN,NaN,3,45.88,Claro Móvel,134825465 ...,NaN,Nao_Validado,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6523966,03248588411,11134220,JADSON WASHINGTON MACIEL DE MELO,51981938392,Nao_Validado,NaN,maciel@hotmail.com,1784,746.76,GEVENUE,363001875138,48984217136,Sim,20/11/2023,NaN,
6523967,18659065816,11503970,FLAVIO DANIEL MICHELLI,17991993918,Nao_Validado,NaN,flaviodanielmichelli@gmail.com,1784,258.73,GEVENUE,006009860175,17991993918,Nao_Validado,NaN,flaviomichelli2013@gmail.com,
6523968,13046637000160,14552992,MARCELO JOSE DA ROCHA - ME,27999770610,Nao_Validado,NaN,chaveiroexpss@hotmail.com,1784,252.53,GEVENUE,566001130130,27995064537,Nao_Validado,NaN,NaN,
6523969,65136594420,14735734,MARIA ELIANE RAMOS DE BRITO,79996001306,Nao_Validado,NaN,britoeliane216@gmail.com,1784,1181.97,GEVENUE,550001986490,79998896985,Nao_Validado,NaN,NaN,


In [2]:

## identificando clientes com mais de um contrato..
base_mais_contratos = base_operacional_hugo.drop_duplicates(["CPF",'BASE'])
base_mais_contratos['duplicado'] = base_mais_contratos.groupby('CPF')['CPF'].transform('count') > 1

## FILTRANDO TUDO QUE FOR VERDADEIRO NA DUPLICIDADE
base_mais_contratos = base_mais_contratos.sort_values(['CPF']).query("duplicado == True")

## INIBINDO CLIENTES GEVENUE
base_mais_contratos = base_mais_contratos.query("BASE != 'GEVENUE'")

## identificando clientes com mais de um contrato..
base_mais_contratos = base_mais_contratos.drop_duplicates(["CPF",'BASE'])
base_mais_contratos['duplicado'] = base_mais_contratos.groupby('CPF')['CPF'].transform('count') > 1
base_mais_contratos = base_mais_contratos.query("duplicado == True").sort_values(['CPF'])

## filtrando apenas a coluna cpf e duplicado para marcar o que for TRUE na base total e poder fazer o mailing apartado.
base_mais_contratos = base_mais_contratos[['CPF','duplicado']].drop_duplicates(['CPF']).reset_index(drop=True)


base_operacional_hugo = pd.merge(base_operacional_hugo, base_mais_contratos, right_on=['CPF'], left_on=['CPF'], how="left")
base_mais_contratos = base_operacional_hugo.query("duplicado == True")
base_mais_contratos = base_mais_contratos.query("BASE != 'GEVENUE'")

base_mais_contratos = base_mais_contratos.sort_values(['CPF'])


### Mantendo na base operacional hugo 
base_operacional_hugo = base_operacional_hugo.query("duplicado.isnull()").reset_index(drop=True)

C:\Users\marco.junior\AppData\Local\Temp\ipykernel_7920\1964028308.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_mais_contratos['duplicado'] = base_mais_contratos.groupby('CPF')['CPF'].transform('count') > 1


In [3]:
### Mailings tel's higienizados clientes com mais contratos
base_mais_contratos_higienizados = base_mais_contratos.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE_HIGIENIZADO']]
base_mais_contratos_higienizados = base_mais_contratos_higienizados.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE_HIGIENIZADO'])

### Tirando os telefones nulos
base_mais_contratos_higienizados = base_mais_contratos_higienizados.query("TELEFONE_HIGIENIZADO.notnull()")

base_mais_contratos_higienizados_0a1785 = base_mais_contratos_higienizados.query("ATRASO <= 1785")
# base_mais_contratos_higienizados_1801a9999 = base_mais_contratos_higienizados.query("ATRASO >= 1801")

In [4]:
### Mailings tel's CONTRATANTE clientes com mais contratos
base_mais_contratos_contratante = base_mais_contratos.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
base_mais_contratos_contratante = base_mais_contratos_contratante.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])

base_mais_contratos_contratante_0a1785 = base_mais_contratos_contratante.query("ATRASO <= 1785")
# base_mais_contratos_contratante_1801a9999 = base_mais_contratos_contratante.query("ATRASO >= 1801")


In [5]:
## MAILINGS PJ CLARO / NET E GEVENUE BB
base_operacional_pj  = base_operacional_hugo[base_operacional_hugo['CPF'].str.len()==14]

# ### SEPARANDO MAILING PJ NET
# net_higi_pj = base_operacional_pj.query("BASE == 'NET' ")
# net_contratante_pj = net_higi_pj.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
# net_higi_pj = net_higi_pj.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE_HIGIENIZADO']]
# net_higi_pj.rename(columns={'TELEFONE_HIGIENIZADO': 'TELEFONE'}, inplace=True)
# net_higi_pj = pd.concat([net_contratante_pj,net_higi_pj])
# net_higi_pj = net_higi_pj.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])
# net_higi_pj = net_higi_pj.query("TELEFONE.notnull()")
# net_higi_pj = net_higi_pj.sample(frac = 1)
# net_higi_pj = net_higi_pj.query("ATRASO <= 2000")


### SEPARANDO MAILING PJ
gevenue_higi_pj = base_operacional_pj.query("BASE == 'GEVENUE'  and ATRASO <= 1785 ")
gevenue_contratante_pj = gevenue_higi_pj.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
gevenue_higi_pj = gevenue_higi_pj.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE_HIGIENIZADO']]

### SEPARANDO MAILING PJ GEVENUE contratante

gevenue_contratante_pj = gevenue_contratante_pj.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])
gevenue_contratante_pj = gevenue_contratante_pj.query("TELEFONE.notnull()")
gevenue_contratante_pj = gevenue_contratante_pj.sample(frac = 1)


### higienziado pj 
gevenue_higi_pj = gevenue_higi_pj.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE_HIGIENIZADO']]
gevenue_higi_pj = gevenue_higi_pj.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE_HIGIENIZADO'])
gevenue_higi_pj = gevenue_higi_pj.query("TELEFONE_HIGIENIZADO.notnull()")
gevenue_higi_pj = gevenue_higi_pj.sample(frac = 1)



### SEPARANDO MAILING PJ CLARO'S
claro_higi_pj = base_operacional_pj.query("BASE != 'GEVENUE' and BASE != 'NET'  and ATRASO <= 1785")
claro_contratante_pj = claro_higi_pj.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
claro_contratante_pj = claro_contratante_pj.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])


claro_higi_pj = claro_higi_pj.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE_HIGIENIZADO']]
claro_higi_pj = claro_higi_pj.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE_HIGIENIZADO'])
claro_higi_pj = claro_higi_pj.query("TELEFONE_HIGIENIZADO.notnull()")
claro_higi_pj = claro_higi_pj.sample(frac = 1)


In [7]:
# # ########################################################### MAILINGS HIGIENIZADOS CLARO  MOVEL ################################################################
### SEPARANDO OS TELEFONES HIGIENIZADOS DA  MOVEL
claro_movel_higi = base_operacional_hugo.query("BASE == 'Claro Móvel' ")\
.loc[:,preditivo_higienizado_claro]\
.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE_HIGIENIZADO'])
claro_movel_higi = claro_movel_higi.query("TELEFONE_HIGIENIZADO.notnull()")
claro_movel_higi = claro_movel_higi.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE_HIGIENIZADO'])


######## MAILING HIGIENIZADO DE -9999 A 200
claro_movel_higi_menor_que_201 = claro_movel_higi.query("ATRASO >= 0 and  ATRASO <= 200 and SALDO_ABERTO >= 50")
claro_movel_higi_menor_que_201 = claro_movel_higi_menor_que_201.sample(frac = 1)

######### MAILING HIGIENIZADO DE 201 A 360
claro_movel_higi_201a360 = claro_movel_higi.query("ATRASO >= 201  and  ATRASO <= 360 and SALDO_ABERTO >= 50")
claro_movel_higi_201a360 = claro_movel_higi_201a360.sample(frac = 1)

######### MAILING HIGIENIZADO DE 361 A 720
claro_movel_higi_361a720 = claro_movel_higi.query("ATRASO >= 361  and  ATRASO <= 720 and SALDO_ABERTO >= 50")
claro_movel_higi_361a720 = claro_movel_higi_361a720.sample(frac = 1)

# #### MAILING HIGIENIZADO DE 721 A 1800
claro_movel_higi_721a1785 = claro_movel_higi.query("ATRASO >= 721 and  ATRASO <= 1785 and SALDO_ABERTO >= 50")
claro_movel_higi_721a1785 = claro_movel_higi_721a1785.sample(frac = 1)


### MAILING HIGIENIZADO DE 1801 A 2500
# claro_movel_higi_1801a2500 = claro_movel_higi.query("ATRASO >= 1801 and  ATRASO <= 2500 and SALDO_ABERTO >= 50")
# claro_movel_higi_1801a2500 = claro_movel_higi_1801a2500.sample(frac = 1)

# # ########################################################### MAILINGS CONTRATANTE CLARO  MOVEL ################################################################
### SEPARANDO OS TELEFONES HIGIENIZADOS DA  MOVEL
claro_movel_intersic = base_operacional_hugo.query("BASE == 'Claro Móvel' ")\
.loc[:,preditivo_telefones_intersic]\
.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])


# # ########################################################### MAILINGS CONTRATANTE CLARO  MOVEL ################################################################
### SEPARANDO OS TELEFONES HIGIENIZADOS DA  MOVEL
claro_movel_intersic = base_operacional_hugo.query("BASE == 'Claro Móvel' ")\
.loc[:,preditivo_telefones_intersic]\
.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])


######### MAILING INTERSIC DE -9999 A 200
claro_movel_intersic = claro_movel_intersic.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])
claro_movel_intersic_0a200 = claro_movel_intersic.query("ATRASO >= 0 and  ATRASO <= 200 and SALDO_ABERTO >= 50")
claro_movel_intersic_0a200 = claro_movel_intersic_0a200.sample(frac = 1)


######### MAILING INTERSIC DE 201 A 360
claro_movel_intersic = claro_movel_intersic.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])
claro_movel_intersic_201a360 = claro_movel_intersic.query("ATRASO >= 201 and  ATRASO <= 360 and SALDO_ABERTO >= 50")
claro_movel_intersic_201a360 = claro_movel_intersic_201a360.sample(frac = 1)


######### MAILING INTERSIC DE 361 A 720
claro_movel_intersic = claro_movel_intersic.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])
claro_movel_intersic_361a720 = claro_movel_intersic.query("ATRASO >= 361 and  ATRASO <= 720 and SALDO_ABERTO >= 50")
claro_movel_intersic_361a720 = claro_movel_intersic_361a720.sample(frac = 1)

# ######### MAILING INTERSIC DE 721 A 1800
claro_movel_intersic = claro_movel_intersic.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])
claro_movel_intersic_721a1785 = claro_movel_intersic.query("ATRASO >= 721 and  ATRASO <= 1785 and SALDO_ABERTO >= 50")
claro_movel_intersic_721a1785 = claro_movel_intersic_721a1785.sample(frac = 1)

######## MAILING INTERSIC DE 1801 A 2500
# claro_movel_intersic = claro_movel_intersic.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])
# claro_movel_intersic_1801a2500 = claro_movel_intersic.query("ATRASO >= 1801 and  ATRASO <= 2500 and SALDO_ABERTO >= 50")
# claro_movel_intersic_1801a2500 = claro_movel_intersic_1801a2500.sample(frac = 1)


# # ######################################################## FILTRANDO AS BASES DE MOVEL HIGIENIZADOS ######################################################
# ### ESTOU FILTRANDO OS HIGIENIZADOS, TUDO QUE FOR DIFERENTE DE VALIDADO SIM PARA NÃO RODAR OS MESMOS NUMEROS NOS MAILINGS DE VALIDADOS
# claro_movel_higi = base_operacional_hugo.query("BASE == 'Claro Movel' and BASE_HIGI_VALIDADOS != 'Sim'")\
# .loc[:,preditivo_higienizado_claro]\
# .drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE_HIGIENIZADO'])

# ############################# MAILINGS CLARO MOVEL HIGIENIZADOS
# movel_0a720_higi = claro_movel_higi.query("ATRASO <= 720")
# movel_0a720_higi = movel_0a720_higi.query("TELEFONE_HIGIENIZADO.notnull()")

# movel_721a9999_higi = claro_movel_higi.query("ATRASO >= 721")
# movel_721a9999_higi = movel_721a9999_higi.query("TELEFONE_HIGIENIZADO.notnull()")

# movel_0a720_higi = movel_0a720_higi.sample(frac = 1)
# movel_721a9999_higi = movel_721a9999_higi.sample(frac = 1)



# # ######################################################## FILTRANDO AS BASES DE MOVEL VALIDADOS ######################################################

# ##### telefones contratante validados
# claro_movel__contratante_validados = base_operacional_hugo.query("BASE_CONTRATANTE_VALIDADOS == 'Sim'  and BASE == 'Claro Movel' ")\
# .loc[:,preditivo_validados_claro]
# ##### telefones higienizados validados
# claro_movel_higi_validados = base_operacional_hugo.query("BASE_HIGI_VALIDADOS == 'Sim'  and BASE == 'Claro Movel' ")\
# .loc[:,preditivo_validados_claro]

# ####################################### COLOCANDO O TEL UM EMBAIXO DO OUTRO PRA REMOVER DUPLICADO E POR NO LAYOUT
# movel_1 = claro_movel__contratante_validados.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
# movel_2 = claro_movel_higi_validados.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE_HIGIENIZADO']]

# movel_2.rename(columns={'TELEFONE_HIGIENIZADO': 'TELEFONE'}, inplace=True)

# claro_movel_validados = pd.concat([movel_1,movel_2])
# claro_movel_validados = claro_movel_validados.drop_duplicates(['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE'])

# ############################# MAILINGS CLARO MOVEL VALIDADOS
# movel_0a720_validados = claro_movel_validados.query("ATRASO <= 720")
# movel_721a9999_validados = claro_movel_validados.query("ATRASO >= 721")
# movel_0a720_validados = movel_0a720_validados.sample(frac = 1)
# movel_721a9999_validados = movel_721a9999_validados.sample(frac = 1)



# # ##########################################################################################################################################################

# ######################################################## FILTRANDO AS BASES DE TV CONTRATANTE (TELEFONES INTERSIC) ###########################################################
### SEPARANDO OS TELEFONES HIGIENIZADOS DA  TV
claro_tv_higi = base_operacional_hugo.query("BASE == 'Claro TV' ")\
.loc[:,preditivo_higienizado_claro]\
.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE_HIGIENIZADO'])
claro_tv_higi = claro_tv_higi.query("TELEFONE_HIGIENIZADO.notnull()")
claro_tv_higi = claro_tv_higi.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE_HIGIENIZADO'])

# ######### MAILING HIGIENIZADO DE 0 A 1350
# claro_tv_higi_0a1350 = claro_tv_higi.query("ATRASO <= 1350  and SALDO_ABERTO >= 50.00 ")
# claro_tv_higi_0a1350 = claro_tv_higi_0a1350.sample(frac = 1)


######### MAILING HIGIENIZADO DE 0 A 1800
claro_tv_higi_0a1785 = claro_tv_higi.query("ATRASO <= 1785  and SALDO_ABERTO >= 50.00 ")
claro_tv_higi_0a1785 = claro_tv_higi_0a1785.sample(frac = 1)


# #### MAILING HIGIENIZADO DE 1801 A 3500
# claro_tv_higi_1801a3500 = claro_tv_higi.query("ATRASO >= 1801 and ATRASO <= 3500  and SALDO_ABERTO >= 50.00 ")
# claro_tv_higi_1801a3500 = claro_tv_higi_1801a3500.sample(frac = 1)


## SEPARANDO A BASE CLARO FIXO, COM OS TELEFONES DO CONTRATANTE (INTERSIC)
claro_tv_contratante = base_operacional_hugo.query("BASE == 'Claro TV' ")\
.loc[:,preditivo_telefones_intersic]

############## MAILING CONTRATANTE DE 0 A 2000
# claro_tv_contratante_0a1350 = claro_tv_contratante.query("ATRASO <= 1350  and SALDO_ABERTO >= 50.00")
# claro_tv_contratante_0a1350 = claro_tv_contratante_0a1350.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])
# claro_tv_contratante_0a1350 = claro_tv_contratante_0a1350.sample(frac = 1)

############## MAILING CONTRATANTE DE 0 A 1800
claro_tv_contratante_0a1785 = claro_tv_contratante.query("ATRASO <= 1785  and SALDO_ABERTO >= 50.00")
claro_tv_contratante_0a1785 = claro_tv_contratante_0a1785.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])
claro_tv_contratante_0a1785 = claro_tv_contratante_0a1785.sample(frac = 1)


############## MAILING CONTRATANTE DE 1801 A 3500 , ESTOU FILTRANDO ATÉ 3500 porque temos mais de 3 milhoes de numeros se considerar até 9999
# claro_tv_contratante_1801a3500 = claro_tv_contratante.query("ATRASO >= 1801 and ATRASO <= 3500  and SALDO_ABERTO >= 50.00")
# claro_tv_contratante_1801a3500 = claro_tv_contratante_1801a3500.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])
# claro_tv_contratante_1801a3500 = claro_tv_contratante_1801a3500.sample(frac = 1)



############## MAILING CONTRATANTE DE 2001 A 2550 , ESTOU FILTRANDO ATÉ 2550 porque temos mais de 3 milhoes de numeros se considerar até 9999
# claro_tv_contratante_1351a2000 = claro_tv_contratante.query("ATRASO >= 1351 and ATRASO <= 2000  and SALDO_ABERTO >= 50.00")
# claro_tv_contratante_1351a2000 = claro_tv_contratante_1351a2000.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])
# claro_tv_contratante_1351a2000 = claro_tv_contratante_1351a2000.sample(frac = 1)




# # ######################################################## FILTRANDO AS TV HIGIENIZADOS QUANDO FOR USAR OS VALIDADOS ######################################################
################ SÓ VAMOS USAR ESSE TRECHO QUANDO FOR SEPARAR OS HIGIENIZADOS E OS VALIDADOS, POIS OS MESMOS NUMEROS NÃO PODEM ESTAR NO MESMO MAILING.
######### CASO NÃO UTILIZE HIGIENIZADO E VALIDADO, O TRECHO ACIMA ATENDE DE UMA FORMA MAIS RESUMIDA E SIMPLES.

# ### ESTOU FILTRANDO OS HIGIENIZADOS, TUDO QUE FOR DIFERENTE DE VALIDADO SIM PARA NÃO RODAR OS MESMOS NUMEROS NOS MAILINGS DE VALIDADOS
# claro_tv_higi = base_operacional_hugo.query("BASE == 'Claro TV' and BASE_HIGI_VALIDADOS != 'Sim'")\
# .loc[:,preditivo_higienizado_claro]\
# .drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE_HIGIENIZADO'])

# ############################# MAILINGS CLARO TV HIGIENIZADOS
# tv_0a720_higi = claro_tv_higi.query("ATRASO <= 2000")
# tv_0a720_higi = tv_0a720_higi.query("TELEFONE_HIGIENIZADO.notnull()")

# tv_721a9999_higi = claro_tv_higi.query("ATRASO >= 2001")
# tv_721a9999_higi = tv_721a9999_higi.query("TELEFONE_HIGIENIZADO.notnull()")

# tv_0a720_higi = tv_0a720_higi.sample(frac = 1)
# tv_721a9999_higi = tv_721a9999_higi.sample(frac = 1)

# # ######################################################## FILTRANDO AS BASES DE TV VALIDADOS ###########################################################

# ##### telefones contratante validados
# claro_tv_contratante_validados = base_operacional_hugo.query("BASE_CONTRATANTE_VALIDADOS == 'Sim'  and BASE == 'Claro TV' ")\
# .loc[:,preditivo_validados_claro]
# ##### telefones higienizados validados
# claro_tv_higi_validados = base_operacional_hugo.query("BASE_HIGI_VALIDADOS == 'Sim'  and BASE == 'Claro TV' ")\
# .loc[:,preditivo_validados_claro]

# ####################################### COLOCANDO O TEL UM EMBAIXO DO OUTRO PRA REMOVER DUPLICADO E POR NO LAYOUT
# tv_1 = claro_tv_contratante_validados.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
# tv_2 = claro_tv_higi_validados.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE_HIGIENIZADO']]
# tv_2.rename(columns={'TELEFONE_HIGIENIZADO': 'TELEFONE'}, inplace=True)

# claro_tv_validados = pd.concat([tv_1,tv_2])
# claro_tv_validados = claro_tv_validados.drop_duplicates(['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE'])

# ############################# MAILINGS CLARO TV VALIDADOS
# tv_0a720_validados = claro_tv_validados.query("ATRASO <= 2000")
# tv_721a9999_validados = claro_tv_validados.query("ATRASO >= 2001")
# tv_0a720_validados = tv_0a720_validados.sample(frac = 1)
# tv_721a9999_validados = tv_721a9999_validados.sample(frac = 1)


##########################################################################################################################################################
######################################################## FILTRANDO AS BASES DE FIXO HIGIENIZADOS ###########################################################

### SEPARANDO OS TELEFONES HIGIENIZADOS DA FIXO.
claro_fixo_higi = base_operacional_hugo.query("BASE == 'Claro Fixo' ")\
.loc[:,preditivo_higienizado_claro]\
.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE_HIGIENIZADO'])
claro_fixo_higi = claro_fixo_higi.query("TELEFONE_HIGIENIZADO.notnull()")
claro_fixo_higi = claro_fixo_higi.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE_HIGIENIZADO'])

######### MAILING HIGIENIZADO DE 0 A 1800
claro_fixo_higi_0a1785 = claro_fixo_higi.query("ATRASO <= 1785")
claro_fixo_higi_0a1785 = claro_fixo_higi_0a1785.sample(frac = 1)

#### MAILING HIGIENIZADO DE 2251 A 99999
# claro_fixo_higi_2251a9999 = claro_fixo_higi.query("ATRASO >= 2251")
# claro_fixo_higi_2251a9999 = claro_fixo_higi_2251a9999.sample(frac = 1)

#### MAILING HIGIENIZADO DE 1801 A 99999
# claro_fixo_higi_1801a99999 = claro_fixo_higi.query("ATRASO >= 1801 and  ATRASO <= 99999")
# claro_fixo_higi_1801a99999 = claro_fixo_higi_1801a99999.sample(frac = 1)


## SEPARANDO A BASE CLARO FIXO, COM OS TELEFONES DO CONTRATANTE (INTERSIC)
claro_fixo_contratante = base_operacional_hugo.query("BASE == 'Claro Fixo' ")\
.loc[:,preditivo_telefones_intersic]

############## MAILING CONTRATANTE DE 0 A 1800
claro_fixo_contratante_0a1785 = claro_fixo_contratante.query("ATRASO <= 1785")
claro_fixo_contratante_0a1785 = claro_fixo_contratante_0a1785.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])
claro_fixo_contratante_0a1785 = claro_fixo_contratante_0a1785.sample(frac = 1)

############## MAILING CONTRATANTE DE 2251 A 999999
# claro_fixo_contratante_2251a9999 = claro_fixo_contratante.query("ATRASO >= 2251")
# claro_fixo_contratante_2251a9999 = claro_fixo_contratante_2251a9999.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])
# claro_fixo_contratante_2251a9999 = claro_fixo_contratante_2251a9999.sample(frac = 1)

# #### MAILING CONTRATANTE DE 1801 A 99999
# claro_fixo_contratante_1801a99999 = claro_fixo_contratante.query("ATRASO >= 1801 and ATRASO <= 99999")
# claro_fixo_contratante_1801a99999 = claro_fixo_contratante_1801a99999.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])
# claro_fixo_contratante_1801a99999 = claro_fixo_contratante_1801a99999.sample(frac = 1)



In [8]:
# # ######################################################## FILTRANDO AS BASES NET  ######################################################

# ### CRIANDO LISTA PARA SEPARAR DDD'S DE SP
# lista_sp = ['11','12','13','14','15','16','17','18','19']

# ############## TELEFONES CONTRATANTE
# base_net_contratante  = base_operacional_hugo.query("BASE == 'NET' ")\
# .loc[:,preditivo_telefones_intersic]\
# .drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE']).sort_values(['CPF'])
# base_net_contratante['DDD'] = base_net_contratante['TELEFONE'].str[0:2]

# ################# MAILING INTERSIC DDD'S SP CONTRATANTE
# base_net_sps_intersic = base_net_contratante.query(f"DDD in{lista_sp}").sort_values(['DDD'])\
# .loc[:,preditivo_telefones_intersic]

# base_net_sp_intersic_0a720 = base_net_sps_intersic.query("ATRASO >= 0 and ATRASO <= 720 and SALDO_ABERTO >= 20.00")
# base_net_sp_intersic_0a720 = base_net_sp_intersic_0a720.sample(frac = 1)

# base_net_sp_intersic_721a2250 = base_net_sps_intersic.query("ATRASO >= 721 and ATRASO <= 2250 and SALDO_ABERTO >= 20.00")
# base_net_sp_intersic_721a2250 = base_net_sp_intersic_721a2250.sample(frac = 1)

# base_net_sp_intersic_2251a9999 = base_net_sps_intersic.query("ATRASO >= 2251 and ATRASO <= 9999 and SALDO_ABERTO >= 20.00")
# base_net_sp_intersic_2251a9999 = base_net_sp_intersic_2251a9999.sample(frac = 1)



# ################# MAILING INTERSIC OUTRAS REGIONAIS 
# base_net_regionais_intersic = base_net_contratante.query(f"DDD not in{lista_sp}").sort_values(['DDD'])\
# .loc[:,preditivo_telefones_intersic]

# base_net_regionais_intersic_0a720 = base_net_regionais_intersic.query("ATRASO >= -99999 and ATRASO <= 720 and SALDO_ABERTO >= 20.00")
# base_net_regionais_intersic_0a720 = base_net_regionais_intersic_0a720.sample(frac = 1)

# base_net_regionais_intersic_721a1080 = base_net_regionais_intersic.query("ATRASO >= 721 and ATRASO <= 1080 and SALDO_ABERTO >= 20.00")
# base_net_regionais_intersic_721a1080 = base_net_regionais_intersic_721a1080.sample(frac = 1)

# base_net_regionais_intersic_1081a1800 = base_net_regionais_intersic.query("ATRASO >= 1081 and ATRASO <= 1800 and SALDO_ABERTO >= 20.00")
# base_net_regionais_intersic_1081a1800 = base_net_regionais_intersic_1081a1800.sample(frac = 1)

# base_net_regionais_intersic_1801a2250 = base_net_regionais_intersic.query("ATRASO >= 1801 and ATRASO <= 2250 and SALDO_ABERTO >= 20.00")
# base_net_regionais_intersic_1801a2250 = base_net_regionais_intersic_1801a2250.sample(frac = 1)


# ############## TELEFONES HIGIENIZADOS
# base_net_higi  = base_operacional_hugo.query("BASE == 'NET' ")\
# .loc[:,preditivo_higienizado_claro]\
# .drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE_HIGIENIZADO'])

# base_net_higi = base_net_higi.query("TELEFONE_HIGIENIZADO != ' ' and TELEFONE_HIGIENIZADO.notnull()")
# #### CRIANDO COLUNA COM O DDD DO TELEFONE PARA FILTRAR NA LISTA DO DDD DE SP
# base_net_higi['DDD'] = base_net_higi['TELEFONE_HIGIENIZADO'].str[0:2]


############################################################### 
################# MAILING HIGIENIZADOS DDD'S SP
# base_net_sp_higi = base_net_higi.query(f"DDD in{lista_sp}").sort_values(['DDD'])\
# .loc[:,preditivo_higienizado_claro]

# base_net_sp_higi_0a720 = base_net_sp_higi.query("ATRASO >= 0 and ATRASO <= 720 and SALDO_ABERTO >= 20.00")
# base_net_sp_higi_0a720 = base_net_sp_higi_0a720.sample(frac = 1)

# base_net_sp_higi_721a2250 = base_net_sp_higi.query("ATRASO >= 721 and ATRASO <= 2250 and SALDO_ABERTO >= 20.00")
# base_net_sp_higi_721a2250 = base_net_sp_higi_721a2250.sample(frac = 1)

# base_net_sp_higi_2251a9999 = base_net_sp_higi.query("ATRASO >= 2251 and ATRASO <= 9999 and SALDO_ABERTO >= 20.00")
# base_net_sp_higi_2251a9999 = base_net_sp_higi_2251a9999.sample(frac = 1)


# ################# MAILING HIGIENIZADOS OUTRAS REGIONAIS 
# base_net_regionais_higi = base_net_higi.query(f"DDD not in{lista_sp}").sort_values(['DDD'])\
# .loc[:,preditivo_higienizado_claro]

# base_net_regionais_higi_0a720 = base_net_regionais_higi.query("ATRASO >= -99999 and ATRASO <= 720 and SALDO_ABERTO >= 20.00")
# base_net_regionais_higi_0a720 = base_net_regionais_higi_0a720.sample(frac = 1)

# base_net_regionais_higi_721a1080 = base_net_regionais_higi.query("ATRASO >= 721 and ATRASO <= 1080 and SALDO_ABERTO >= 20.00")
# base_net_regionais_higi_721a1080 = base_net_regionais_higi_721a1080.sample(frac = 1)

# base_net_regionais_higi_1081a1800 = base_net_regionais_higi.query("ATRASO >= 1081 and ATRASO <= 1800 and SALDO_ABERTO >= 20.00")
# base_net_regionais_higi_1081a1800 = base_net_regionais_higi_1081a1800.sample(frac = 1)

# base_net_regionais_higi_1801a2250 = base_net_regionais_higi.query("ATRASO >= 1801 and ATRASO <= 2250 and SALDO_ABERTO >= 20.00")
# base_net_regionais_higi_1801a2250 = base_net_regionais_higi_1801a2250.sample(frac = 1)



################ SÓ VAMOS USAR ESSE TRECHO QUANDO FOR SEPARAR OS HIGIENIZADOS E OS VALIDADOS, POIS OS MESMOS NUMEROS NÃO PODEM ESTAR NO MESMO MAILING.
######### CASO NÃO UTILIZE HIGIENIZADO E VALIDADO, O TRECHO ACIMA ATENDE DE UMA FORMA MAIS RESUMIDA E SIMPLES.

# net_higi = base_operacional_hugo.query("BASE == 'NET' and BASE_HIGI_VALIDADOS != 'Sim'")\
# .loc[:,preditivo_higienizado_claro]\
# .drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE_HIGIENIZADO'])

# ############################# MAILINGS NET HIGIENIZADOS

# net_0a2000_higi = net_higi.query("ATRASO <= 2000")
# ### filtrando somente as linhas de telefones higienizados que não são nulos.
# net_0a2000_higi = net_0a2000_higi.query("TELEFONE_HIGIENIZADO.notnull()")

# net_2001a9999_higi = net_higi.query("ATRASO >= 2001")
# net_2001a9999_higi = net_2001a9999_higi.query("TELEFONE_HIGIENIZADO.notnull()")


# net_0a2000_higi = net_0a2000_higi.sample(frac = 1)
# net_2001a9999_higi = net_2001a9999_higi.sample(frac = 1)


# ########################################################### MAILINGS  NET VALIDADOS ################################################################

# ##### telefones contratante validados
# net_contratante_validados = base_operacional_hugo.query("BASE_CONTRATANTE_VALIDADOS == 'Sim'  and BASE == 'NET' ")\
# .loc[:,preditivo_validados_claro]
# ##### telefones higienizados validados
# net_higi_validados = base_operacional_hugo.query("BASE_HIGI_VALIDADOS == 'Sim'  and BASE == 'NET' ")\
# .loc[:,preditivo_validados_claro]

# ####################################### COLOCANDO O TEL UM EMBAIXO DO OUTRO PRA REMOVER DUPLICADO E POR NO LAYOUT
# net_1 = net_contratante_validados.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
# net_2 = net_higi_validados.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE_HIGIENIZADO']]

# net_2.rename(columns={'TELEFONE_HIGIENIZADO': 'TELEFONE'}, inplace=True)

# net_validados = pd.concat([net_1,net_2])
# net_validados = net_validados.drop_duplicates(['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE'])

# ##########################################################################################################################################################
# ######################################################## FILTRANDO AS BASES DE NET VALIDADOS ###########################################################

# net_0a2000_validados = net_validados.query("ATRASO <= 2000")
# net_2001a9999_validados = net_validados.query("ATRASO >= 2001")

# net_0a2000_validados = net_0a2000_validados.sample(frac = 1)
# net_2001a9999_validados = net_2001a9999_validados.sample(frac = 1)


# #########################################################################################################################################################
# ######################################################## FILTRANDO AS BASES GEVENUE HIGIENIZADOS ######################################################
### SEPARANDO OS TELEFONES HIGIENIZADOS DA GEVENUE.

gevenue_higi = base_operacional_hugo.query("BASE == 'GEVENUE' ")\
.loc[:,preditivo_higienizado_claro]\
.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE_HIGIENIZADO'])
gevenue_higi = gevenue_higi.query("TELEFONE_HIGIENIZADO.notnull()")
gevenue_higi = gevenue_higi.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE_HIGIENIZADO'])

######### MAILING HIGIENIZADO DE 0 A 360
gevenue_higi_0a360 = gevenue_higi.query("ATRASO >= 0 and ATRASO <= 360 and SALDO_ABERTO >= 50 ")
gevenue_higi_0a360 = gevenue_higi_0a360.sample(frac = 1)

#### MAILING HIGIENIZADO DE 361 A 720
gevenue_higi_361a720 = gevenue_higi.query("ATRASO >= 361 and ATRASO <= 720 and SALDO_ABERTO >= 50")
gevenue_higi_361a720 = gevenue_higi_361a720.sample(frac = 1)

#### MAILING GEVENUE  HIGIENIZADO DE 721 A 1080
gevenue_higi_721a1080 = gevenue_higi.query("ATRASO >= 721 and ATRASO <= 1080 and SALDO_ABERTO >= 50")
gevenue_higi_721a1080 = gevenue_higi_721a1080.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE_HIGIENIZADO'])
gevenue_higi_721a1080 = gevenue_higi_721a1080.sample(frac = 1)

#### MAILING GEVENUE  HIGIENIZADO DE 1081 A 1800
gevenue_higi_1081a1785 = gevenue_higi.query("ATRASO >= 1081 and ATRASO <= 1785 and SALDO_ABERTO >= 50")
gevenue_higi_1081a1785 = gevenue_higi_1081a1785.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE_HIGIENIZADO'])
gevenue_higi_1081a1785 = gevenue_higi_1081a1785.sample(frac = 1)

#### MAILING GEVENUE  HIGIENIZADO DE 1801 A 3000
# gevenue_higi_1801a3000 = gevenue_higi.query("ATRASO >= 1801 and ATRASO <= 3000 and SALDO_ABERTO >= 50")
# gevenue_higi_1801a3000 = gevenue_higi_1801a3000.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE_HIGIENIZADO'])
# gevenue_higi_1801a3000 = gevenue_higi_1801a3000.sample(frac = 1)




## SEPARANDO A BASE GEVENUE, COM OS TELEFONES DO CONTRATANTE (GEVENUE)
gevenue_contratante = base_operacional_hugo.query("BASE == 'GEVENUE' ")\
.loc[:,preditivo_telefones_intersic]

############## MAILING CONTRATANTE DE 0 A 360
gevenue_contratante_0a360 = gevenue_contratante.query("ATRASO >= 0 and SALDO_ABERTO <= 360 ")
gevenue_contratante_0a360 = gevenue_contratante_0a360.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])
gevenue_contratante_0a360 = gevenue_contratante_0a360.sample(frac = 1)

############## MAILING CONTRATANTE DE 361 A 720
gevenue_contratante_361a720 = gevenue_contratante.query("ATRASO >= 361 and ATRASO <= 720 and SALDO_ABERTO >= 50")
gevenue_contratante_361a720 = gevenue_contratante_361a720.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])
gevenue_contratante_361a720 = gevenue_contratante_361a720.sample(frac = 1)

############## MAILING CONTRATANTE DE 721 A 1080
gevenue_contratante_721a1080 = gevenue_contratante.query("ATRASO >= 721 and ATRASO <= 1080 and SALDO_ABERTO >= 50")
gevenue_contratante_721a1080 = gevenue_contratante_721a1080.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])
gevenue_contratante_721a1080 = gevenue_contratante_721a1080.sample(frac = 1)


############## MAILING CONTRATANTE DE 1081 A 1800
gevenue_contratante_1081a1785 = gevenue_contratante.query("ATRASO >= 1081  and ATRASO <= 1785 and SALDO_ABERTO >= 50")
gevenue_contratante_1081a1785 = gevenue_contratante_1081a1785.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])
gevenue_contratante_1081a1785 = gevenue_contratante_1081a1785.sample(frac = 1)

############## MAILING CONTRATANTE DE 1801 A 3000
# gevenue_contratante_1801a3000 = gevenue_contratante.query("ATRASO >= 1801 and ATRASO <= 3000 and SALDO_ABERTO >= 50")
# gevenue_contratante_1801a3000 = gevenue_contratante_1801a3000.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE'])
# gevenue_contratante_1801a3000 = gevenue_contratante_1801a3000.sample(frac = 1)



############### SÓ VAMOS USAR ESSE TRECHO QUANDO FOR SEPARAR OS HIGIENIZADOS E OS VALIDADOS, POIS OS MESMOS NUMEROS NÃO PODEM ESTAR NO MESMO MAILING.
######## CASO NÃO UTILIZE HIGIENIZADO E VALIDADO, O TRECHO ACIMA ATENDE DE UMA FORMA MAIS RESUMIDA E SIMPLES.

claro_gevenue_higi = base_operacional_hugo.query("BASE == 'GEVENUE' and BASE_HIGI_VALIDADOS != 'Sim'")\
.loc[:,preditivo_higienizado_claro]\
.drop_duplicates(['CPF','NOME_x','DEVEDOR','TELEFONE_HIGIENIZADO'])

############################# MAILINGS GEVENUE
gevenue_0a1785_higi = claro_gevenue_higi.query("ATRASO <= 1785")
### FILTRANDO APENAS OS NÃO NULOS PARA NAO SUBIR CLIENTE COM TELEFONE VAZIO 
gevenue_0a1785_higi = gevenue_0a1785_higi.query("TELEFONE_HIGIENIZADO.notnull()")

# gevenue_2001a9999_higi = claro_gevenue_higi.query("ATRASO >= 2001")
### FILTRANDO APENAS OS NÃO NULOS PARA NAO SUBIR CLIENTE COM TELEFONE VAZIO 
# gevenue_2001a9999_higi = gevenue_2001a9999_higi.query("TELEFONE_HIGIENIZADO.notnull()")

gevenue_0a1785_higi = gevenue_0a1785_higi.sample(frac = 1)
# gevenue_2001a9999_higi = gevenue_2001a9999_higi.sample(frac = 1)


########################################################### MAILINGS  GEVENUE VALIDADOS ################################################################

##### telefones contratante validados
gevenue_contratante_validados = base_operacional_hugo.query("BASE_CONTRATANTE_VALIDADOS == 'Sim'  and BASE == 'GEVENUE' ")\
.loc[:,preditivo_validados_claro]
##### telefones higienizados validados
gevenue_higi_validados = base_operacional_hugo.query("BASE_HIGI_VALIDADOS == 'Sim'  and BASE == 'GEVENUE' ")\
.loc[:,preditivo_validados_claro]

####################################### COLOCANDO O TEL UM EMBAIXO DO OUTRO PRA REMOVER DUPLICADO E POR NO LAYOUT
gevenue_1 = gevenue_contratante_validados.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE']]
gevenue_2 = gevenue_higi_validados.loc[:,['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE_HIGIENIZADO']]

gevenue_2.rename(columns={'TELEFONE_HIGIENIZADO': 'TELEFONE'}, inplace=True)

gevenue_validados = pd.concat([gevenue_1,gevenue_2])
gevenue_validados = gevenue_validados.drop_duplicates(['CPF', 'NOME_x', 'DEVEDOR', 'BASE', 'FILA', 'ATRASO', 'SALDO_ABERTO','TELEFONE'])

############################# MAILINGS GEVENUE VALIDADOS
gevenue_0a1785_validados = gevenue_validados.query("ATRASO <= 1785")
# gevenue_2001a9999_validados = gevenue_validados.query("ATRASO >= 2001")

In [9]:
# #########################################################################################################################################################
# #################################################################### BAIXANDO OS MAILINGS  ##################################################################

# ##################### MOVEL
claro_movel_higi_menor_que_201.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\HIGIENIZADOS/MOVEL_HIGI_0A200_1112.csv", index=False,sep=';')
claro_movel_higi_201a360.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\HIGIENIZADOS/MOVEL_HIGI_201A360_1112.csv", index=False,sep=';')
claro_movel_higi_361a720.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\HIGIENIZADOS/MOVEL_HIGI_361A720_1112.csv", index=False,sep=';')
claro_movel_higi_721a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\HIGIENIZADOS/MOVEL_HIGI_721a1785_1112.csv", index=False,sep=';')
# claro_movel_higi_1801a2500.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\HIGIENIZADOS/MOVEL_HIGI_1801a2500_3011.csv", index=False,sep=';')

claro_movel_intersic_0a200.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\INTERSIC/MOVEL_INTERSIC_0a200_1112.csv", index=False,sep=';')
claro_movel_intersic_201a360.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\INTERSIC/MOVEL_INTERSIC_201a360_1112.csv", index=False,sep=';')
claro_movel_intersic_361a720.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\INTERSIC/MOVEL_INTERSIC_361a720_1112.csv", index=False,sep=';')
claro_movel_intersic_721a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\INTERSIC/MOVEL_INTERSIC_721a1785_1112.csv", index=False,sep=';')
# claro_movel_intersic_1801a2500.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\INTERSIC/MOVEL_INTERSIC_1801a2500_3011.csv", index=False,sep=';')


##################### TV
claro_tv_higi_0a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\HIGIENIZADOS/TV_0a1785_HIGI_1112.csv", index=False,sep=';')
# claro_tv_higi_1801a3500.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\HIGIENIZADOS/TV_1801a3500_HIGI_3011.csv", index=False,sep=';')

claro_tv_contratante_0a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\INTERSIC/TV__0a1785_INTERSIC_1112.csv", index=False,sep=';')
# # claro_tv_contratante_1801a3500.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\INTERSIC/TV_1801a3500_INTERSIC_3011.csv", index=False,sep=';')


###################### FIXO
claro_fixo_higi_0a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\HIGIENIZADOS/FIXO_0A1785_HIGI_1112.csv", index=False,sep=';')
# claro_fixo_higi_1801a99999.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\HIGIENIZADOS/FIXO1801a99999_HIGI_3011.csv", index=False,sep=';')
# claro_fixo_higi_2251a9999.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\HIGIENIZADOS/FIXO_2251A9999_HIGI_2108.csv", index=False,sep=';')

claro_fixo_contratante_0a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\INTERSIC/FIXO_0a1785_INTERSIC_1112.csv", index=False,sep=';')
# claro_fixo_contratante_1801a99999.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\INTERSIC/FIXO_1801a99999_INTERSIC_3011.csv", index=False,sep=';')
# claro_fixo_contratante_2251a9999.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\INTERSIC/FIXO_2251A9999_INTERSIC_1610.csv", index=False,sep=';')


############## MAILINGS CLIENTES COM MAIS DE UM CONTRATO

base_mais_contratos_contratante_0a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\MAIS DE UM CONTRATO/base_mais_contratos_contratante_0a1785_1112.csv", index=False,sep=';')
# base_mais_contratos_contratante_1801a9999.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\MAIS DE UM CONTRATO/base_mais_contratos_contratante_1801a9999_3011.csv", index=False,sep=';')
base_mais_contratos_higienizados_0a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\MAIS DE UM CONTRATO/base_mais_contratos_higienizados_0a1785_1112.csv", index=False,sep=';')
# base_mais_contratos_higienizados_1801a9999.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\CLARO\MAIS DE UM CONTRATO/base_mais_contratos_higienizados_1801a9999_3011.csv", index=False,sep=';')


# # ###################### NET

# base_net_sp_intersic_0a720.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\NET\INTERSIC/NET_INTERSIC_SP_0A720_2308.csv", index=False,sep=';')
# base_net_sp_intersic_721a2250.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\NET\INTERSIC/NET_INTERSIC_SP_721A2250_2308.csv", index=False,sep=';')
# base_net_sp_intersic_2251a9999.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\NET\INTERSIC/NET_INTERSIC_SP_2251A9999_2308.csv", index=False,sep=';')
# base_net_regionais_intersic_0a720.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\NET\INTERSIC/NET_INTERSIC_REGIONAIS_0a720_1109.csv", index=False,sep=';')
# base_net_regionais_intersic_721a1080.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\NET\INTERSIC/NET_INTERSIC_REGIONAIS_721a1080_1109.csv", index=False,sep=';')
# base_net_regionais_intersic_1081a1800.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\NET\INTERSIC/NET_INTERSIC_REGIONAIS_1081a1800_1109.csv", index=False,sep=';')
# base_net_regionais_intersic_1801a2250.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\NET\INTERSIC/NET_INTERSIC_REGIONAIS_1801a2250_1109.csv", index=False,sep=';')


# # # base_net_sp_higi_0a720.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\NET\HIGIENIZADOS/NET_HIGI_SP_0A720_2308.csv", index=False,sep=';')
# # # base_net_sp_higi_721a2250.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\NET\HIGIENIZADOS/NET_HIGI_SP_721A2250_2308.csv", index=False,sep=';')
# # # base_net_sp_higi_2251a9999.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\NET\HIGIENIZADOS/NET_HIGI_SP_2251A9999_2308.csv", index=False,sep=';')
# base_net_regionais_higi_0a720.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\NET\HIGIENIZADOS/NET_HIGI_REGIONAIS_0A720_1109.csv", index=False,sep=';')
# base_net_regionais_higi_721a1080.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\NET\HIGIENIZADOS/NET_HIGI_REGIONAIS_721A1080_1109.csv", index=False,sep=';')
# base_net_regionais_higi_1081a1800.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\NET\HIGIENIZADOS/NET_HIGI_REGIONAIS_1081A1800_1109.csv", index=False,sep=';')
# base_net_regionais_higi_1801a2250.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\NET\HIGIENIZADOS/NET_HIGI_REGIONAIS_1801A2250_1109.csv", index=False,sep=';')


# # # ###################### GEVENUE

gevenue_higi_0a360.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\GEVENUE\HIGIENIZADOS/GEVENUE_HIGI_0A360_1112.csv", index=False,sep=';')
gevenue_higi_361a720.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\GEVENUE\HIGIENIZADOS/GEVENUE_HIGI_361A720_1112.csv", index=False,sep=';')
gevenue_higi_721a1080.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\GEVENUE\HIGIENIZADOS/GEVENUE_HIGI_721A1080_1112.csv", index=False,sep=';')
gevenue_higi_1081a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\GEVENUE\HIGIENIZADOS/GEVENUE_HIGI_1081A1785_1112.csv", index=False,sep=';')
# gevenue_higi_1801a3000.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\GEVENUE\HIGIENIZADOS/GEVENUE_HIGI_1801a3000_3011.csv", index=False,sep=';')

gevenue_contratante_0a360.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\GEVENUE\CONTRATANTE/GEVENUE_CONTRATANTE_0A360_1112.csv", index=False,sep=';')
gevenue_contratante_361a720.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\GEVENUE\CONTRATANTE/GEVENUE_CONTRATANTE_361A720_1112.csv", index=False,sep=';')
gevenue_contratante_721a1080.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\GEVENUE\CONTRATANTE/GEVENUE_CONTRATANTE_721A1080_1112.csv", index=False,sep=';')
gevenue_contratante_1081a1785.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\GEVENUE\CONTRATANTE/GEVENUE_CONTRATANTE_1081A1785_1112.csv", index=False,sep=';')
# gevenue_contratante_1801a3000.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\GEVENUE\CONTRATANTE/GEVENUE_CONTRATANTE_1801A3000_3011.csv", index=False,sep=';')


In [10]:
# MAILINGS DE PJ 
# net_higi_pj.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\MAILINGS PJ/NET_HIGI_PJ_1109.csv", index=False,sep=';')
gevenue_higi_pj.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\MAILINGS PJ/GEVENUE_HIGI_PJ_1112.csv", index=False,sep=';')
claro_higi_pj.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\MAILINGS PJ/CLARO_HIGI_PJ_1112.csv", index=False,sep=';')

# net_intersic_pj.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Novembro\9\Jorgin da Clock\EMAILS_TELECOB/NET_INTERSIC_PJ_2811.csv", index=False,sep=';')
gevenue_contratante_pj.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\MAILINGS PJ/GEVENUE_INTERSIC_PJ_1112.csv", index=False,sep=';')
claro_contratante_pj.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\MAILINGS DA SEMANA\MAILINGS PJ/CLARO_INTERSIC_PJ_1112.csv", index=False,sep=';')


### RETORNO DOS HIGIENIZADOS QUE A OPERAÇÃO PEDE:

In [2]:
###### BAIXANDO OS EMAILS DA GEVENUE HIGIENIZADOS (DAIANA PEDE PRA AÇÃO MASSIVA)

## SEPARANDO OS EMAILS HIGIENIZADOS 
emails_gevenue_higienizados = base_operacional_hugo.query("BASE == 'GEVENUE' and  EMAIL_HIGIENIZADO.notnull()").loc[:,['CPF','NOME_x','EMAIL_HIGIENIZADO','ATRASO','BASE','CONTRATO']]\
.drop_duplicates(['CPF','NOME_x','EMAIL_HIGIENIZADO'])

##### SEPARANDO OS EMAILS DO CONTRATANTE
emails_gevenue_contratante = base_operacional_hugo.query("BASE == 'GEVENUE' and  EMAIL.notnull()").loc[:,['CPF','NOME_x','EMAIL','ATRASO','BASE','CONTRATO']]\
.drop_duplicates(['CPF','NOME_x','EMAIL'])


###RENOMEANDO PARA CONCATENAR OS DOIS HIGIENIZADOS E CONTRATANTE
emails_gevenue_higienizados.rename(columns={'EMAIL_HIGIENIZADO': 'EMAIL'}, inplace=True)

# CONCATENANDO, REMOVENDO DUPLICADO E RESETANDO O INDEX
emails_gevenue_devolver = pd.concat([emails_gevenue_higienizados,emails_gevenue_contratante])
emails_gevenue_devolver = emails_gevenue_devolver.drop_duplicates(['CPF','NOME_x','EMAIL']).reset_index(drop=True)



###### FILTRANDO O QUE É = CPF (11 DIGITOS) PORQUE A QUANTIDDE ULTRAPASSA 1 MILHÃO DE LINHAS
### ORDENANDO POR ATRASO DO MENOR PARA O MAIOR E CPF , FILTRANDO SOMENTE AS LINHAS DE EMAIL QUE NÃO ESTÃO VAZIAS.
## DIVIDINDO EM 2 ARQUIVOS PARA ABRIREM EM EXCEL
emails_gevenue_devolver_cpf = emails_gevenue_devolver[emails_gevenue_devolver['CPF'].str.len()==11].reset_index(drop=True)
emails_gevenue_devolver_cpf = emails_gevenue_devolver_cpf.sort_values(['ATRASO','CPF']).query("EMAIL !=' '")
emails_gevenue_cpf_1 = emails_gevenue_devolver_cpf.iloc[0:999999]
emails_gevenue_cpf_2 = emails_gevenue_devolver_cpf.iloc[1000000:2000000]
emails_gevenue_cpf_3 = emails_gevenue_devolver_cpf.iloc[2000001:3000000]

###### FILTRANDO O QUE É = CNPJ (14 DIGITOS).
#### ORDENANDO POR ATRASO DO MENOR PARA O MAIOR E CPF , FILTRANDO SOMENTE AS LINHAS DE EMAIL QUE NÃO ESTÃO VAZIAS.
emails_gevenue_devolver_cnpj = emails_gevenue_devolver[emails_gevenue_devolver['CPF'].str.len()==14].reset_index(drop=True)
emails_gevenue_devolver_cnpj = emails_gevenue_devolver_cnpj.sort_values(['ATRASO','CPF']).query("EMAIL !=' '")



##### BAIXANDO O ARQUIVO COM 3 ABAS, DUAS CONTENDO EMAILS DE CLIENTES PESSOA FISICA E UMA COM PESSOA JURIDICA.
writer = pd.ExcelWriter(r'R:\TI\TELEFONIA\Mailings Prontos\2023\Dezembro\7\Markin 7\EMAILS_TELECOB/Emails_Gevenue_0712.xlsx', engine='xlsxwriter')

#Salvando o Relatorio na pasta.
emails_gevenue_cpf_1.to_excel(writer, sheet_name='Emails_Gevenue_Cpf_1', index=False)
emails_gevenue_cpf_2.to_excel(writer, sheet_name='Emails_Gevenue_Cpf_2', index=False)
emails_gevenue_cpf_3.to_excel(writer, sheet_name='Emails_Gevenue_Cpf_3', index=False)
emails_gevenue_devolver_cnpj.to_excel(writer, sheet_name='Emails_Gevenue_Cnpj', index=False)
writer.save()

In [4]:
#### BAIXANDO OS EMAILS DA CLARO MOVEL, FIXO E TV
emails_claro = base_operacional_hugo.loc[:,['CPF','NOME_x','EMAIL_HIGIENIZADO','ATRASO','BASE','CONTRATO']]\
.query("EMAIL_HIGIENIZADO.notnull() and CONTRATO.notnull() ").drop_duplicates(['CPF','EMAIL_HIGIENIZADO']).reset_index(drop=True)
## Filtrando tudo exceto Gevenue
emails_claro = emails_claro.query("BASE != 'GEVENUE'")

emails_claro_1 = emails_claro[:1000000]
emails_claro_2 = emails_claro[1000001:2000000]


##### BAIXANDO O ARQUIVO COM 3 ABAS, DUAS CONTENDO EMAILS DE CLIENTES PESSOA FISICA E UMA COM PESSOA JURIDICA.
writer = pd.ExcelWriter(r'R:\TI\TELEFONIA\Mailings Prontos\2023\Dezembro\7\Markin 7/Emails_Claro_0712.xlsx', engine='xlsxwriter')

#Salvando o Relatorio na pasta.
emails_claro_1.to_excel(writer, sheet_name='emails_CLAROS_FULL_1', index=False)
emails_claro_2.to_excel(writer, sheet_name='emails_CLAROS_FULL_2', index=False)
writer.save()

<!-- ### Analise da base operacional -->

In [ ]:
# Criando coluna Faixa de atraso com suas respectivas faixas
base_operacional_hugo.loc[base_operacional_hugo.ATRASO.between(-1000, -1),    'Faixa_de_Atraso'] = "Atraso (Menor que 0)"
base_operacional_hugo.loc[base_operacional_hugo.ATRASO.between(0, 2000),    'Faixa_de_Atraso'] = "Atraso (0 a 2000)"
base_operacional_hugo.loc[base_operacional_hugo.ATRASO.between(2001, 2250),    'Faixa_de_Atraso'] = "Atraso (2001 a 2250)"
base_operacional_hugo.loc[base_operacional_hugo.ATRASO.between(2501, 3500),    'Faixa_de_Atraso'] = "Atraso (2501 a 3500)"
base_operacional_hugo.loc[base_operacional_hugo.ATRASO.between(3501, 999999),    'Faixa_de_Atraso'] = "Atraso (Acima 3501)"


###################################################### TELEFONES INTERSIC POR BASE E FAIXA DE ATRASO #####################################################

# FILTRAMOS PRIMEIRO OS TELEFONES DO CONTRATANTE NÃO NULOS, PARA GARANTIR A CONTAGEM DAS LINHAS COM TELEFONE DE FATO.
# SEPARAMOS APENAS AS COLUNAS PARA A ANALISE DE BASE / TELEFONES INTERSIC POR FAIXA DE ATRASO.
# REMOVEMOS OS DUPLICADOS POR CPF/NOME/TELEFONE, POIS RECEBEMOS MUITOS TELEFONES DO MESMO CPF, E EFETUANDO OS CRUZAMENTOS PARA UNIR TUDO OS CLIENTES FICAM DUPLICADOS, O QUE MUDA O NUMERO REAL.
## POR FIM AGRUPO BASE (CARTEIRA) POR FAIXA DE ATRASO, PRA SABER A QUANTIDADE DE TELEFONE DO CONTRATANTE POR CPF 
analitic_telefones_intersic = base_operacional_hugo.query("TELEFONE.notnull()").loc[:,['BASE','Faixa_de_Atraso','CPF','NOME_x','TELEFONE']]\
    .drop_duplicates(['CPF','NOME_x','TELEFONE'])\
    .groupby(['BASE','Faixa_de_Atraso'])['TELEFONE'].count().reset_index().sort_values(['TELEFONE'])


## pula linha no print
'\n'

###################################################### TELEFONES HIGIENIZADOS POR BASE E FAIXA DE ATRASO #####################################################

# FILTRAMOS PRIMEIRO OS HIGIENIZADOS NÃO NULOS, PORQUE NEM TODOS OS CPFS ENVIADOS PRA HIGIENIZAR A AFINDER TEM TELEFONE
# SEPARAMOS APENAS AS COLUNAS PARA A ANALISE DE BASE / TELEFONE HIGIENIZADO POR FAIXA DE ATRASO.
# REMOVEMOS OS DUPLICADOS POR CPF/NOME/TELEFONE, POIS RECEBEMOS MUITOS TELEFONES DO MESMO CPF, E EFETUANDO OS CRUZAMENTOS PARA UNIR TUDO OS CLIENTES FICAM DUPLICADOS, O QUE MUDA O NUMERO REAL.
## POR FIM AGRUPO BASE (CARTEIRA) POR FAIXA DE ATRASO, PRA SABER A QUANTIDADE DE TELEFONE HIGIENIZADO POR CPF 
analitic_telefones_higienizados = base_operacional_hugo.query("TELEFONE_HIGIENIZADO.notnull()").loc[:,['BASE','Faixa_de_Atraso','CPF','NOME_x','TELEFONE_HIGIENIZADO']]\
    .drop_duplicates(['CPF','NOME_x','TELEFONE_HIGIENIZADO'])\
    .groupby(['BASE','Faixa_de_Atraso'])['TELEFONE_HIGIENIZADO'].count().reset_index().sort_values(['TELEFONE_HIGIENIZADO'])


## pula linha no print
'\n'

# ######################### ANÁLISE DE DÉBITO POR FAIXA DE ATRASO

####### SEPARAMOS AS COLUNAS NECESSARIAS PARA ANALISAR POR FAIXA DE ATRASO O SALDO ABERTO DO CLIENTE
##### REMOVEMOS OS DUPLICADOS POR CPF E ATRASO, PARA QUE SE O CLIENTE TIVER MAIS DE UMA FATURA E CASO ENTRE EM UMA OUTRA FAIXA DE ATRASO SEJA CALCULADO O NUMERO VERDADEIRO
analitic_SaldoPorCpf = base_operacional_hugo.loc[:,['BASE','Faixa_de_Atraso','CPF','ATRASO','SALDO_ABERTO']].drop_duplicates(['CPF','ATRASO'])
### FAÇO UM REPLACE NA VIRGULA PARA PONTO, POIS SÓ ASSIM CONSIGO CONVERTER A COLUNA SALDO_ABERTO PARA FLOAT
# analitic_SaldoPorCpf['SALDO_ABERTO'] = analitic_SaldoPorCpf['SALDO_ABERTO'].str.replace('.', '').str.replace(',', '.').astype(float)

analitic_SaldoPorCpf['SALDO_ABERTO'] = analitic_SaldoPorCpf['SALDO_ABERTO'].astype('Float64')

# Criando coluna Faixa de atraso com suas respectivas faixas
analitic_SaldoPorCpf.loc[analitic_SaldoPorCpf.SALDO_ABERTO.between(-99999, 19.99),    'Faixa_de_Débito'] = "Saldo (Menor que 20,00)"
analitic_SaldoPorCpf.loc[analitic_SaldoPorCpf.SALDO_ABERTO.between(20.0, 9999999),    'Faixa_de_Débito'] = "Saldo (Maior que 20,00)"

analitic_SaldoPorCpf_group = analitic_SaldoPorCpf.groupby(['BASE','Faixa_de_Atraso','Faixa_de_Débito'])['CPF'].count().reset_index()

##### JUNTANDO OS TELEFONES DO CONTRATANTE E OS HIGIENIZADOS 
analitic_full_bases = pd.merge(analitic_telefones_intersic, analitic_telefones_higienizados, right_on=['BASE','Faixa_de_Atraso'], left_on=['BASE','Faixa_de_Atraso'], how="inner")


In [ ]:
analitic_telefones_bases_atraso = pd.pivot_table(analitic_full_bases, values=['TELEFONE','TELEFONE_HIGIENIZADO'], index=['BASE'],
                       columns=['Faixa_de_Atraso']).fillna(0)
analitic_telefones_bases_atraso

In [ ]:
analitic_SaldoPorCpf_group = pd.pivot_table(analitic_SaldoPorCpf_group, values=['CPF'], index=['BASE','Faixa_de_Débito'],
                       columns=['Faixa_de_Atraso']).fillna(0)

In [ ]:
# Carregar o arquivo Excel
writer = pd.ExcelWriter(r'R:\TI\TELEFONIA\Mailings Prontos\2023\Agosto\8\Analitic_Telecobs_Hugo_0808.xlsx', engine='xlsxwriter')

# Salvar a tabela pivotada "analitic_SaldoPorCpf_group" na coluna B, linhas 2 a 11
analitic_SaldoPorCpf_group.to_excel(writer, sheet_name='Full_Bases', startrow=1, startcol=1)

# Salvar a tabela pivotada "analitic_telefones_bases_atraso" na coluna B, linhas 12 a 20
analitic_telefones_bases_atraso.to_excel(writer, sheet_name='Full_Bases', startrow=12, startcol=1)

# Fechar o escritor do Excel
writer.save()
